# Test freezoing:

1. Tracker 6D
2. print x,px
3. new tracker, same line, freeze variables
3. print x,px from the first tracker, see if changes.


In [1]:
import json
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import clear_output
import sys
import rich
import pickle


import xobjects as xo
import xtrack as xt
import xpart as xp

sys.path.append('../')
import BBStudies.Tracking.XsuitePlus as xPlus
import BBStudies.Tracking.InteractionPoint as inp
import BBStudies.Physics.Detuning as dtune
import BBStudies.Physics.Base as phys
import BBStudies.Plotting.BBPlots as bbplt
import BBStudies.Physics.Constants as cst


LINEFOLDER   = '../Data/Lines/'
SURVEYFOLDER = '../Data/Surveys/'
df_columns = ['name', 's', 'x', 'px', 'y', 'py', 'zeta', 'delta', 'ptau', 'betx', 'bety', 'alfx', 'alfy', 'gamx', 'gamy', 'dx', 'dpx', 'dy', 'dpy', 'mux', 'muy', 'qx', 'qy', 'qs', 'dqx', 'dqy']


line   = {}
twiss  = {}
survey = {}

# Importing survey
for seq in ['lhcb1','lhcb2']:
    _beam = seq[-2:]
    survey[_beam] = pd.read_pickle(SURVEYFOLDER + f'survey_opticsfile32_{seq}.pkl')

# Importing twiss
for seq in ['lhcb1','lhcb4']:
    _beam = seq[-2:]
    
    # Importing Line
    line[_beam] = xPlus.importLine(LINEFOLDER + f'line_opticsfile32_{seq}.json')
    
    # Importing twiss and tracker
    if _beam == 'b1':
        tracker = xt.Tracker(line=line[_beam])
        xtwiss  = tracker.twiss()
    else:
        xtwiss  = xt.Tracker(line=line[_beam]).twiss()
    
    twiss[_beam]= pd.DataFrame(xtwiss,columns = df_columns)
    twiss[_beam].set_index('name',inplace=True)


# Creating twiss b2 from b4
#==========================================
twiss['b2'] = xPlus.twiss_b2_from_b4(twiss['b4'])



# Clearing xsuite ouput
clear_output(wait=False)






# Filtering twiss (Needed for IP objects)
#====================================
twiss['b1'] = xPlus.filter_twiss(twiss['b1'],entries=['drift','..'])
twiss['b2'] = xPlus.filter_twiss(twiss['b2'],entries=['drift','..','bbcw'])
#====================================
B1 = inp.Beam('b1',twiss['b1'],survey['b1'],
              Nb       = 1.15e11,
              E        = 6.8e12,
              emittx_n = 2.5e-6,
              emitty_n = 2.5e-6,
              dp_p0    = 0)
    
B2 = inp.Beam('b2',twiss['b2'],survey['b2'],
              Nb       = 1.15e11,
              E        = 6.8e12,
              emittx_n = 2.5e-6,
              emitty_n = 2.5e-6,
              dp_p0    = 0)

IP1 = inp.InteractionPoint('ip1',B1,B2)
IP5 = inp.InteractionPoint('ip5',B1,B2)


beam = 'b1'
# Deactivating all wires
#===================================================
tracker.vars['enable_qff'] = 0
for IP in ['ip1','ip5']:
    tracker.vars[f"bbcw_rw_{IP}.{beam}"] = 1
    tracker.vars[f"bbcw_i_{IP}.{beam}"]  = 0

    
# Creating BB knobs
#===================================================
allVars = list(tracker.vars._owner.keys())
allElements = list(tracker.element_refs._owner.keys())


for _ip in ['ip1','ip5','ip2','ip8']:
    bb_lr = [name for name in allElements if ('bb_lr' in name)&(f'{_ip[-1]}b1' in name)]
    bb_ho = [name for name in allElements if ('bb_ho' in name)&(f'{_ip[-1]}b1' in name)]

    # New knob:
    tracker.vars[f'{_ip}_bblr_ON'] = 1
    tracker.vars[f'{_ip}_bbho_ON'] = 1

    # Linking to new knob 
    for _lr in bb_lr:
        # Infividual knobs
        tracker.vars[f'{_lr}_ON'] = 1
        tracker.element_refs[_lr].other_beam_num_particles = tracker.vars[f'{_lr}_ON']*tracker.vars[f'{_ip}_bblr_ON']*tracker.element_refs[_lr].other_beam_num_particles._value
        


    for _ho in bb_ho:
        tracker.element_refs[_ho].slices_other_beam_num_particles = tracker.vars[f'{_ip}_bbho_ON']*tracker.element_refs[_ho].slices_other_beam_num_particles._value[0]

# Creating sext and oct knobs
#====================================================
ks = [name for name in allVars if ('ksf' in name)|('ksd' in name)]
ko = [name for name in allVars if ('kof.a' in name)|('kod.a' in name)]

tracker.vars['all_oct_ON']  = 1
tracker.vars['all_sext_ON'] = 1
for _ks in ks:
    if tracker.vars[_ks]._expr is None:
        tracker.vars[_ks] = tracker.vars['all_sext_ON']*tracker.vars[_ks]._value
    else:
        tracker.vars[_ks] = tracker.vars['all_sext_ON']*tracker.vars[_ks]._expr 
for _ko in ko:
    if tracker.vars[_ko]._expr is None:
        tracker.vars[_ko] = tracker.vars['all_oct_ON']*tracker.vars[_ko]._value
    else:
        tracker.vars[_ko] = tracker.vars['all_oct_ON']*tracker.vars[_ko]._expr 




In [3]:
tracker.vars['on_bb_charge']._info()


#  vars['on_bb_charge']._get_value()
   vars['on_bb_charge'] = 1.0

#  vars['on_bb_charge']._expr is None

#  vars['on_bb_charge'] does not influence any target



In [4]:
bbON = pd.read_parquet('/home/phbelang/abp/BBStudies/Data/Mask/final_twiss_BBON.parquet')
bbOFF = pd.read_parquet('/home/phbelang/abp/BBStudies/Data/Mask/final_twiss_BBOFF.parquet')

In [11]:
tracker.vars['on_bb_charge'] = 0

In [12]:
tracker.line['bb_ho.r5b1_05'].to_dict()

{'__class__': 'BeamBeamBiGaussian2D',
 'ref_shift_x': 0.0,
 'ref_shift_y': 0.0,
 'other_beam_shift_x': -0.05234059730162516,
 'other_beam_shift_y': -4.800945452756805e-07,
 'post_subtract_px': 8.160118841759895e-10,
 'post_subtract_py': -7.195487430546343e-15,
 'other_beam_q0': 1.0,
 'other_beam_beta0': 0.9999999904805956,
 'other_beam_num_particles': 110000000000.0,
 'other_beam_Sigma_11': 1.0563197212870904e-06,
 'other_beam_Sigma_13': 0.0,
 'other_beam_Sigma_33': 1.3477085576597237e-06,
 'min_sigma_diff': 1e-10}

In [13]:
tracker.line['bb_ho.r5b1_05'].to_dict()

{'__class__': 'BeamBeamBiGaussian3D',
 '_sin_phi': 0.00016000492843582529,
 '_cos_phi': 0.9999999871992113,
 '_tan_phi': 0.00016000493048401457,
 '_sin_alpha': -4.902601323383437e-05,
 '_cos_alpha': 0.999999998798225,
 'ref_shift_x': 1.0940772201370274e-05,
 'ref_shift_px': 0.00016001938586576855,
 'ref_shift_y': 1.3566669641812923e-09,
 'ref_shift_py': -3.237792154796969e-08,
 'ref_shift_zeta': 6.37502058305008e-07,
 'ref_shift_pzeta': 4.523862442429305e-07,
 'other_beam_shift_x': -2.1884696252527647e-05,
 'other_beam_shift_px': 0.0,
 'other_beam_shift_y': 1.7550394052352341e-09,
 'other_beam_shift_py': 0.0,
 'other_beam_shift_zeta': 0.0,
 'other_beam_shift_pzeta': 0.0,
 'post_subtract_x': -6.025385810952411e-16,
 'post_subtract_px': 1.7207297537628202e-07,
 'post_subtract_y': 4.8320582662897665e-20,
 'post_subtract_py': -1.3799362297519522e-11,
 'post_subtract_zeta': 0.0,
 'post_subtract_pzeta': 2.7709390337804507e-11,
 'other_beam_q0': 1.0,
 'num_slices_other_beam': 1,
 'slices_othe

In [ ]:
tracker.element_refs[]

In [10]:
tracker.line['bb_lr.r5b1_25'].other_beam_q0

1.0

In [5]:
bbON

,name,keyword,s,betx,alfx,mux,bety,alfy,muy,x,...,sig54,sig55,sig56,sig61,sig62,sig63,sig64,sig65,sig66,n1
,,,,,,,,,,,,,,,,,,,,,
ip3:1,ip3:1,marker,0.0000,128.276740,2.467975,0.000000,209.832693,-2.524309,0.000000,1.523046e-07,...,3.264346e-11,0.006613,3.697706e-08,3.535379e-09,8.613150e-11,-4.737576e-09,-6.142732e-11,3.697706e-08,1.056824e-08,0.0
drift_0:0,drift_0:0,drift,21.0300,48.921087,1.305476,0.042764,331.543175,-3.263163,0.012703,2.247739e-07,...,3.264343e-11,0.006613,3.697707e-08,5.346723e-09,8.613150e-11,-6.029392e-09,-6.142732e-11,3.697707e-08,1.056824e-08,0.0
mcbwv.4l3.b2:1,mcbwv.4l3.b2:1,vkicker,21.0300,48.921087,1.305476,0.042764,331.543175,-3.263163,0.012703,2.247739e-07,...,3.264343e-11,0.006613,3.697707e-08,5.346723e-09,8.613150e-11,-6.029392e-09,-6.142732e-11,3.697707e-08,1.056824e-08,0.0
drift_1:0,drift_1:0,drift,22.5205,45.152270,1.223084,0.047812,341.348713,-3.315530,0.013408,2.299101e-07,...,3.264343e-11,0.006613,3.697707e-08,5.475102e-09,8.613150e-11,-6.120949e-09,-6.142732e-11,3.697707e-08,1.056824e-08,0.0
bpmw.4l3.b2:1,bpmw.4l3.b2:1,monitor,22.5205,45.152270,1.223084,0.047812,341.348713,-3.315530,0.013408,2.299101e-07,...,3.264343e-11,0.006613,3.697707e-08,5.475102e-09,8.613150e-11,-6.120949e-09,-6.142732e-11,3.697707e-08,1.056824e-08,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
bpmw.4r3.b2:1,bpmw.4r3.b2:1,monitor,26636.4387,266.908177,3.708664,62.281307,114.217681,-1.735758,60.289271,7.449596e-08,...,3.264349e-11,0.006613,3.697706e-08,1.602202e-09,8.613150e-11,-3.358872e-09,-6.142732e-11,3.697706e-08,1.056824e-08,0.0
drift_30796:0,drift_30796:0,drift,26640.1832,239.909076,3.501675,62.283662,127.709378,-1.867314,60.294206,8.742003e-08,...,3.264349e-11,0.006613,3.697706e-08,1.924721e-09,8.613150e-11,-3.588886e-09,-6.142732e-11,3.697706e-08,1.056824e-08,0.0
mcbwh.4r3.b2:1,mcbwh.4r3.b2:1,hkicker,26640.1832,239.909076,3.501675,62.283662,127.709378,-1.867314,60.294206,8.742003e-08,...,3.264349e-11,0.006613,3.697706e-08,1.924721e-09,8.613150e-11,-3.588886e-09,-6.142732e-11,3.697706e-08,1.056824e-08,0.0
